## Update a wflow model: gauging stations

Once you have a **Wflow** model, you may want to update your model in order to use a new landuse map, change a parameter value, add sample locations, use different forcing data, create and run different scenarios etc.

With HydroMT, you can easily read your model and update one or several components of your model using the **update** function of the command line interface (CLI). Here are the steps and some examples on how to **update/add gauging stations** to your model.

All lines in this notebook which starts with ! are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste them in your shell to get more feedback.

### Import packages

In this notebook, we will use some functions of HydroMT to plot the new gauging stations of the updated model. Here are the libraries to import to realize these steps.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

In [ ]:
# for plotting
import matplotlib.pyplot as plt
from matplotlib import colors, patheffects, cm
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
proj = ccrs.PlateCarree() # plot projection

In [ ]:
# import hydromt
import hydromt
from hydromt.models import MODELS

In [ ]:
# setup logging
from  hydromt.log import setuplog
logger = setuplog("update_model_gauges", log_level=10)

### Locations of the gauging stations

Adding gauging stations to your **Wflow** model can be quite useful. It allows you to directly extract model outputs at specific locations. For example for model validation, calibration, you can then directly compare model results to available observations.

In our previous notebook, we built a wflow model in which the locations of the available [GRDC](https://www.bafg.de/GRDC/EN/Home/homepage_node.html) stations are including. If you have other observation data available, you can easily add them to your model using hydroMT. One way to do it is to prepare a csv table containing the ID, latitude and longitude of the available stations. An example **test_stations.csv** has been prepared. Let's have a look at it. 

In [ ]:
fn_station = 'test_stations.csv'
with open(fn_station, 'r', encoding='utf8') as f:
    txt = f.read()
print(txt)

Here we can see that we have defined three stations with an ID and the longitude (x) and latitude (y) coordinates, in the same EPSG system as our model (EPSG 4326). We can now add them to our model.

### hydroMT CLI update interface

Using the `hydroMT build` API, we can update one or several components of an already existing Wflow model. Let's get an overview of the available options:

In [ ]:
# Print the options available from the update command
! hydromt update --help

### Update wflow gauging stations

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
! hydromt update wflow wflow_piave_subbasin -o ./wflow_piave_gauges -c setup_gauges --opt source_name=test_stations.csv --opt basename=test-flow -v

The example above means the following: run **hydromt** with:

- `update wflow`: i.e. update a wflow model
- `wflow_piave_subbasin`: original model folder
- `-o ./wflow_piave_gauges`: output updated model folder
- `-c setup_gauges`: model component to update, here setup_gauges to add gauging stations
- `--opt gauges_fn=test_stations.csv`: name in hydromt data catalog or path to a csv file containing the gauges locations
- `--opt basename=test-flow`: name of the new gauges map in the staticmaps of Wflow
- `v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

Other available options are available in the [docs(setup_gauges)](https://deltares.github.io/hydromt_wflow/latest/generated/hydromt_wflow.wflow.WflowModel.setup_gauges.html#hydromt_wflow.wflow.WflowModel.setup_gauges).

### Update wflow gauging stations without snapping

By default, the **setup_gauges** of hydroMT Wflow will assume that gauges are for flow sampling and will therefore snap the gauges locations to the river. If your observations are used to compare rainfall data, then you do not want hydroMT to move the location of your gauges. This is done by setting `--opt snap_to_river=False`.

By default, hydroMT Wflow will update the TOML in order to save both river discharge and precipitation at the new gauges locations. With our rainfall stations, we only need to save the precipitation. We can set this with hydroMT using the options:

- `--opt gauge_toml_header=["P-station"]`: set the column name in the outputcsv file of the Wflow run in the TOML configuration
- `--opt gauge_toml_param=["vertical.precipitation"]`: set the Wflow variable to save in the corresponding outputcsv file column of Wflow run in the TOML configuration 

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
! hydromt update wflow wflow_piave_gauges -c setup_gauges --opt source_name=test_stations.csv --opt basename=test-rain --opt snap_to_river=False --opt gauge_toml_header=["P-station"] --opt gauge_toml_param=["vertical.precipitation"] -v

The example above means the following: run **hydromt** with:

- `update wflow`: i.e. update a wflow model
- `wflow_piave_gauges`: model folder to update (here we save our updates in the same model folder)
- `-c setup_gauges`: model component to update, here setup_gauges to add gauging stations
- `--opt gauges_fn=test_stations.csv`: name in hydromt data catalog or path to a csv file containing the gauges locations
- `--opt basename=test-rain`: name of the new gauges map in the staticmaps of Wflow
- `--opt snap_to_river=False`
- `--opt gauge_toml_header=["P-station"]`: set the column name in the outputcsv file of the Wflow run in the TOML configuration
- `--opt gauge_toml_param=["vertical.precipitation"]`: set the Wflow variable to save in the corresponding outputcsv file column of Wflow run in the TOML configuration 
- `v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

Other available options are available in the [docs(setup_gauges)](https://deltares.github.io/hydromt_wflow/latest/generated/hydromt_wflow.wflow.WflowModel.setup_gauges.html#hydromt_wflow.wflow.WflowModel.setup_gauges).

Note: when changing several options, the command line can get quite long. You can also create a hydroMT configuration file where you set all the different options and use it in your command line with the `-i` argument instead of `-c` and several `--opt`

### Visualization of the gauges

We can now plot our newly created gauges stations maps and check the differences between the flow and rain maps.